In [1]:
from data.data import load_data
from model.ddpm import DDPM
from model.sde import VESDE
from pc_sampling import sampler
from loss import JDAMLoss
from eval_metrics import PSNR, SSIM

import tensorflow as tf

In [2]:
data = load_data()

In [3]:
model = DDPM(activation=tf.keras.activations.swish)

In [4]:
sde = VESDE(
    pet_score_func=lambda x, t: model(x, t, training=True),
    mri_score_func=lambda x, t: model(x, t, training=True),
    )